In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
%load_ext autoreload
# # the "1" means: always reload modules marked with "%aimport"
%autoreload 2

from __future__ import absolute_import, division, print_function
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os, sys
# from tqdm import tqdm
# import warnings

sns.set_context("poster", font_scale=1.3)
pd.set_option('display.max_columns', 100)

# add the data functions to the path
src_data_dir = os.path.join(os.getcwd(), os.pardir, 'src/data')
sys.path.append(src_data_dir)

# functions to load the data
import dataset as ds

In [2]:
filename = os.path.join(os.getenv('HOME'), 'Dropbox', 'C4SF-datasci-homeless', 'processed', 'homeless_row_per_enrollment.csv')
df = pd.read_csv(filename, header=0, index_col=0, parse_dates=['entry_date', 'exit_date', 'residential_move_in_date'])

/Users/mattmollison/miniconda3/envs/py3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,race,ethnicity,gender,veteran_status,project_entry_id,client_age_at_entry,last_permanent_zip,entry_date,exit_date,project_id,housing_status_project_start,living_situation_before_program_entry,client_location,household_id,disabling_condition,continuously_homeless_one_year,times_homeless_past_three_years,months_homeless_this_time,chronic_homeless,in_permanent_housing,residential_move_in_date,domestic_violence_victim,months_ago_dv_occurred,dv_currently_fleeing,days_enrolled,days_to_residential_move_in,head_of_household,noncash_benefit,disability_type,project_name,project_type_code,address_city,address_postal_code
Personal ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
214224,White,Non-Hispanic/Non-Latino,Male,False,203474,60,00000,2012-01-01,2013-09-08,2938,Category 1 - Homeless,Hotel or motel paid for without emergency shel...,NaN,203474,True,False,NaN,NaN,False,False,NaT,False,NaN,False,616,NaN,False,None,Physical,SCz - HSC - Page Smith Community House,Transitional housing,Santa Cruz,95060
194592,White,Non-Hispanic/Non-Latino,Male,True,150135,53,91103,2012-01-02,2013-08-31,2134,Category 1 - Homeless,"Staying or living in a family member's room, a...",NaN,150135,True,False,NaN,NaN,False,False,NaT,False,NaN,False,607,NaN,False,Supplemental Nutrition Assistance Program (Foo...,Mental Health Problem,MOSBE Veterans Transition Center - Outreach,Other,Marina,93933
179384,White,Hispanic/Latino,Female,False,155873,50,93245,2012-01-03,2016-01-28,2086,Category 1 - Homeless,Place not meant for habitation,CA-506,155873,False,False,NaN,NaN,False,False,NaT,False,NaN,False,1486,NaN,True,None,None,MOSBE The Salvation Army - Good Samaritan Center,Services Only,NaN,93955
183019,White,Hispanic/Latino,Male,False,155872,28,93950,2012-01-03,2012-10-26,2086,Category 1 - Homeless,Place not meant for habitation,CA-506,155872,False,False,NaN,NaN,False,False,NaT,False,NaN,False,297,NaN,False,None,None,MOSBE The Salvation Army - Good Samaritan Center,Services Only,NaN,93955
182885,Black or African American,Non-Hispanic/Non-Latino,Male,False,153810,48,93911,2012-01-04,2013-05-02,2086,Category 1 - Homeless,"Emergency shelter, including hotel or motel pa...",NaN,153810,False,False,NaN,NaN,False,False,NaT,False,NaN,False,484,NaN,False,None,None,MOSBE The Salvation Army - Good Samaritan Center,Services Only,NaN,93955


In [4]:
df = df.sort_values(by=['entry_date'])

In [5]:
# number the enrollments
df['enrollment_num'] = df[['entry_date']].groupby(level=0).cumcount().values + 1

In [6]:
max_enrollments = 3

choose_these = (df['enrollment_num'].groupby(level=0).max() <= max_enrollments)

df_max = df.loc[choose_these[choose_these].index, :]

In [7]:
df.shape

(58209, 34)

In [8]:
df_max.shape

(14473, 34)

In [9]:
n_projects = 3

field = 'project_type_code'

for n in range(1, n_projects+1):
    if n == 1:
        cols = [
            'race',
            'ethnicity',
            'gender',
            'veteran_status',
            'project_entry_id',
            'client_age_at_entry',
            'entry_date',
            'housing_status_project_start',
            ]

        cols.append(field)
        
        df_out = df_max[df_max['enrollment_num'] == n][cols]
        df_out = df_out.rename(columns={field: 'project_{}'.format(n)})
    else:
        cols = [field]

        df_enroll = df_max[df_max['enrollment_num'] == n][cols]
        df_enroll = df_enroll.rename(columns={field: 'project_{}'.format(n)})
        
        df_out = df_out.merge(df_enroll, how='left', left_index=True, right_index=True)
    
perm = df_max[df_max['enrollment_num'] <= n_projects]['in_permanent_housing'].groupby(level=0).max()

df_out['in_permanent_housing'] = perm

In [10]:
df_out.head()

,race,ethnicity,gender,veteran_status,project_entry_id,client_age_at_entry,entry_date,housing_status_project_start,project_1,project_2,project_3,in_permanent_housing
Personal ID,,,,,,,,,,,,
173781,White,Hispanic/Latino,Female,False,252608,34,2014-02-28,At-risk of homelessness,Street Outreach,Emergency Shelter,NaN,False
173782,White,Hispanic/Latino,Male,False,314084,10,2014-05-06,Category 1 - Homeless,Emergency Shelter,NaN,NaN,False
173783,White,Hispanic/Latino,Female,False,314085,12,2014-05-06,Category 1 - Homeless,Emergency Shelter,NaN,NaN,False
173803,White,Hispanic/Latino,Female,False,201286,32,2013-02-08,Category 1 - Homeless,Emergency Shelter,NaN,NaN,False
173804,White,Hispanic/Latino,Female,False,201288,11,2013-02-08,Category 1 - Homeless,Emergency Shelter,NaN,NaN,False


In [11]:
filename = os.path.join(os.getenv('HOME'), 'Dropbox', 'C4SF-datasci-homeless', 'processed', 'homeless_3_projects_outcome.csv')

df_out.to_csv(filename)